In [1]:
%load_ext autotime

In [2]:
import pandas as pd
import numpy as np
import implicit as imp
import scipy.sparse as sp
import time

time: 2.97 s


In [3]:
np.__config__.show()

lapack_opt_info:
    libraries = ['mkl_intel_lp64', 'mkl_intel_thread', 'mkl_core', 'iomp5', 'pthread']
    library_dirs = ['/home/hatice/Development/anaconda2/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/home/hatice/Development/anaconda2/include']
blas_opt_info:
    libraries = ['mkl_intel_lp64', 'mkl_intel_thread', 'mkl_core', 'iomp5', 'pthread']
    library_dirs = ['/home/hatice/Development/anaconda2/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/home/hatice/Development/anaconda2/include']
openblas_lapack_info:
  NOT AVAILABLE
lapack_mkl_info:
    libraries = ['mkl_intel_lp64', 'mkl_intel_thread', 'mkl_core', 'iomp5', 'pthread']
    library_dirs = ['/home/hatice/Development/anaconda2/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/home/hatice/Development/anaconda2/include']
blas_mkl_info:
    libraries = ['mkl_intel_lp64', 'mkl_intel_thread', 

In [4]:
data_file = pd.read_table('data/training.csv', sep = ',', header=None, engine='python')
print(data_file.shape)
movie_file = pd.read_table('ml-1m/movies.dat', sep = '::', header=None, engine='python')
print(movie_file.shape)

(702811, 4)
(3883, 3)
time: 2.84 s


In [5]:
#movies 3666(gercege karşılık gelen index) alıp 3952(gerçekid) döner, movie_indices 3952 alıp 3666 döner
users = np.unique(data_file[0]) # 1(0.idex) den 6040(6039.index) a kadar
movies = np.unique(movie_file[0]) # 1(0.idex) den 3952(3666.idex) ye kadar

number_of_rows = len(users) #6040
number_of_columns = len(movies) #3667

movie_indices, user_indices = {}, {}
 
for i in range(len(movies)):
    movie_indices[movies[i]] = i # movie_indices[3952] = 3666 x.filmin indisini verir
    
for i in range(len(users)):
    user_indices[users[i]] = i # x.userın indisini verir

time: 32.3 ms


In [6]:
V = sp.lil_matrix((number_of_rows, number_of_columns))
for line in data_file.values:
    u, i , r , gona = map(int,line)
    V[user_indices[u], movie_indices[i]] = r # gerçek user ve movie idnin indexini bulup ratingi matrixteki yere atar

time: 4.41 s


In [7]:
W = V>0.5
W[W == True] = 1
W[W == False] = 0
# To be consistent with our V matrix
W = W.astype(np.float64)


/home/hatice/Development/anaconda2/lib/python2.7/site-packages/scipy/sparse/compressed.py:213: SparseEfficiencyWarning: Comparing a sparse matrix with 0 using == is inefficient, try using != instead.
  ", try using != instead.", SparseEfficiencyWarning)
/home/hatice/Development/anaconda2/lib/python2.7/site-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


time: 5.53 s


In [8]:
lambda_ = 0.1
n_factors = 16
m, n = V.shape
n_iterations = 20
X = 5 * np.random.rand(m, n_factors) 
Y = 5 * np.random.rand(n_factors, n)

time: 3.7 ms


In [9]:
def get_error(Q, X, Y, W):
    return np.sum((W * (Q - np.dot(X, Y)))**2)

time: 97.8 ms


In [10]:
errors = []
for ii in range(n_iterations):
    X = np.linalg.solve(np.dot(Y, Y.T) + lambda_ * np.eye(n_factors), 
                        np.dot(Y, V.T)).T
    Y = np.linalg.solve(np.dot(X.T, X) + lambda_ * np.eye(n_factors),
                        np.dot(X.T, V))
    if ii % 100 == 0:
        print('{}th iteration is completed'.format(ii))
    errors.append(get_error(V, X, Y, W))
Q_hat = np.dot(X, Y)
print('Error of rated movies: {}'.format(get_error(V, X, Y, W)))

AttributeError: 'module' object has no attribute 'solve_triangular'

time: 281 ms


In [ ]:
alsModel = imp.als.AlternatingLeastSquares(factors=50)
alsModel.fit(V)
print(alsModel)

In [ ]:
lambd = 0.1
factors = 16
iteration = 50
def als(matrix):
    X = 5 * np.random.rand(matrix.shape[0], factors) 
    Y = 5 * np.random.rand(factors, matrix.shape[1])
    for ii in range(iteration):
        print(ii, X, time.ctime())
        X = np.linalg.solve(np.dot(Y, Y.T) + lambd * np.eye(factors), np.dot(Y, matrix.T)).T
        print(ii, Y, time.ctime())
        Y = np.linalg.solve(np.dot(X.T, X) + lambd * np.eye(factors),np.dot(X.T, matrix))
    return (X,Y)

In [ ]:
result = als(V)